# Caso 3 – MDVRP Robusto con CBC y HiGHS

## Objetivo del Notebook

Este notebook implementa el **Caso 3** (Multi-Depot Vehicle Routing Problem) del proyecto LogistiCo, utilizando **únicamente solvers de código abierto** (CBC y HiGHS).

### ¿Por qué un modelo ligero?

El Caso 3 involucra 50-100 clientes y múltiples depósitos, lo que genera un espacio de soluciones muy grande para solvers gratuitos. En máquinas limitadas (como Google Colab), los solvers comerciales como Gurobi no están disponibles, y los solvers abiertos pueden tener dificultades para encontrar soluciones óptimas en tiempo razonable.

### Estrategia de Robustez

Para **garantizar factibilidad** en lugar de optimalidad, implementamos:

1. **Filtrado KNN de arcos**: Reducimos el espacio de búsqueda considerando solo los K vecinos más cercanos para cada cliente, reduciendo el número de arcos de O(n²) a O(n·k).

2. **Modelo relajado para solución inicial**: Primero resolvemos un modelo simplificado que ignora restricciones de capacidad y flujo, garantizando que cada cliente sea visitado.

3. **Pipeline CBC→HiGHS**: 
   - **CBC** (COIN-OR Branch and Cut): Rápido para encontrar soluciones factibles con gap amplio (10%)
   - **HiGHS**: Refinamiento de la solución CBC con gap más ajustado (2%)

### Restricciones Relajadas

- Espacialmente: Solo consideramos arcos KNN (no todos los pares posibles)
- Temporalmente: Límites de tiempo más generosos en los solvers
- Optimality gap: Aceptamos soluciones sub-óptimas si son factibles

**Enunciado oficial**: `data/Caso3/README.md`


## 2. Imports y Configuración


In [ ]:
# Imports básicos
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
import csv
import time
from collections import defaultdict

# Pyomo para modelado de optimización
from pyomo.environ import *

# KNN para filtrado de arcos
try:
    from sklearn.neighbors import NearestNeighbors
    SKLEARN_AVAILABLE = True
    print("✓ sklearn disponible para KNN")
except ImportError:
    SKLEARN_AVAILABLE = False
    print("⚠ sklearn no disponible, usando fallback de distancia")

print("✓ Imports completados")


## 3. Carga de Datos

Cargamos los datos desde `data/Caso3/`:
- **clients.csv**: Clientes con demandas y restricciones de vehículos
- **depots.csv**: Centros de distribución con capacidades
- **vehicles.csv**: Flota disponible con capacidades y autonomías
- **parameters_urban.csv**: Parámetros de costos

**Referencia**: Ver `data/Caso3/README.md` para descripción completa de columnas


In [ ]:
def cargar_datos_caso3(ruta_base="data/Caso3/"):
    """Carga todos los archivos del Caso 3 y calcula matrices de distancia/tiempo"""
    
    # Cargar archivos
    clientes = pd.read_csv(ruta_base + "clients.csv")
    depositos = pd.read_csv(ruta_base + "depots.csv")
    vehiculos = pd.read_csv(ruta_base + "vehicles.csv")
    parametros = pd.read_csv(ruta_base + "parameters_urban.csv")
    
    # Convertir parámetros a diccionario
    params = {}
    for _, row in parametros.iterrows():
        params[row['Parameter']] = row['Value']
    
    # Crear diccionario unificado con coordenadas
    nodos = {}
    
    # Agregar depósitos
    for _, depot in depositos.iterrows():
        nodos[depot['StandardizedID']] = {
            'lat': depot['Latitude'],
            'lon': depot['Longitude'],
            'tipo': 'deposito',
            'capacidad': depot['Capacity']
        }
    
    # Agregar clientes
    for _, cliente in clientes.iterrows():
        nodos[cliente['StandardizedID']] = {
            'lat': cliente['Latitude'],
            'lon': cliente['Longitude'],
            'tipo': 'cliente',
            'demanda': cliente['Demand'],
            'restriccion': cliente['VehicleSizeRestriction']
        }
    
    # Calcular matriz de distancias (Haversine)
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371  # Radio de la Tierra en km
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        return R * c
    
    nodos_ids = list(nodos.keys())
    n = len(nodos_ids)
    distancias = np.zeros((n, n))
    
    for i, id1 in enumerate(nodos_ids):
        for j, id2 in enumerate(nodos_ids):
            if i != j:
                distancias[i][j] = haversine(
                    nodos[id1]['lat'], nodos[id1]['lon'],
                    nodos[id2]['lat'], nodos[id2]['lon']
                )
    
    # Calcular matriz de tiempos (asumiendo 30 km/h promedio urbano)
    tiempos = distancias / 30.0  # horas
    
    # Eficiencia de combustible por tipo de vehículo (promedio)
    fuel_efficiency = {
        'small van': (params['fuel_efficiency_van_small_min'] + params['fuel_efficiency_van_small_max']) / 2,
        'medium van': (params['fuel_efficiency_van_medium_min'] + params['fuel_efficiency_van_medium_max']) / 2,
        'light truck': (params['fuel_efficiency_truck_light_min'] + params['fuel_efficiency_truck_light_max']) / 2
    }
    
    # Agregar eficiencia a vehículos
    vehiculos['FuelEfficiency'] = vehiculos['VehicleType'].map(fuel_efficiency)
    
    return {
        'clientes': clientes,
        'depositos': depositos,
        'vehiculos': vehiculos,
        'parametros': params,
        'nodos': nodos,
        'nodos_ids': nodos_ids,
        'distancias': distancias,
        'tiempos': tiempos
    }

# Cargar datos
print("Cargando datos del Caso 3...")
datos = cargar_datos_caso3()

print(f"\n📊 Resumen de datos:")
print(f"  • Clientes: {len(datos['clientes'])}")
print(f"  • Depósitos: {len(datos['depositos'])}")
print(f"  • Vehículos: {len(datos['vehiculos'])}")
print(f"  • Nodos totales: {len(datos['nodos_ids'])}")
print(f"\n💰 Parámetros de costo:")
print(f"  • Costo fijo: ${datos['parametros']['C_fixed']:,.0f} COP/vehículo")
print(f"  • Costo distancia: ${datos['parametros']['C_dist']:,.0f} COP/km")
print(f"  • Costo tiempo: ${datos['parametros']['C_time']:,.0f} COP/hora")
print(f"  • Precio combustible: ${datos['parametros']['fuel_price']:,.0f} COP/galón")

# Mostrar muestra de clientes
print(f"\n👥 Muestra de clientes:")
print(datos['clientes'][['StandardizedID', 'Demand', 'VehicleSizeRestriction']].head(5))


## 4. Preprocesamiento y Validación de Grafos

El preprocesamiento es crítico para la eficiencia:
- **KNN (K-Nearest Neighbors)**: Limitamos los arcos a los K vecinos más cercanos
- **Validación de autonomía**: Solo consideramos arcos que el vehículo puede recorrer
- **Validación de restricciones**: Respetamos las restricciones de tipo de vehículo por cliente


In [ ]:
def obtener_knn_pairs(coords, k_neighbors=6):
    """
    Construye pares de vecinos más cercanos usando KNN
    
    Args:
        coords: numpy array de coordenadas (N x 2)
        k_neighbors: número de vecinos más cercanos
    
    Returns:
        set de tuplas (i, j) representando arcos permitidos
    """
    if SKLEARN_AVAILABLE:
        # Usar sklearn si está disponible
        knn = NearestNeighbors(n_neighbors=min(k_neighbors + 1, len(coords)), metric='euclidean')
        knn.fit(coords)
        distances, indices = knn.kneighbors(coords)
        
        knn_pairs = set()
        for i, neighbors in enumerate(indices):
            for j in neighbors:
                if i != j:
                    knn_pairs.add((i, j))
        return knn_pairs
    else:
        # Fallback: calcular distancias manualmente
        n = len(coords)
        knn_pairs = set()
        
        for i in range(n):
            distances = []
            for j in range(n):
                if i != j:
                    dist = np.sqrt((coords[i][0] - coords[j][0])**2 + (coords[i][1] - coords[j][1])**2)
                    distances.append((dist, j))
            
            distances.sort()
            for _, j in distances[:k_neighbors]:
                knn_pairs.add((i, j))
        
        return knn_pairs

print("✓ Función KNN definida")


In [ ]:
def validar_previo(datos, k_neighbors=8, knn_autonomy_margin=1.2):
    """
    Valida que el grafo sea conectado y construye arcos factibles por vehículo
    
    Args:
        datos: diccionario con todos los datos cargados
        k_neighbors: número de vecinos KNN
        knn_autonomy_margin: margen de autonomía (1.2 = 20% extra)
    
    Returns:
        dict con feasible_arcs_per_vehicle y report
    """
    
    print(f"\n🔍 Validando grafo con k_neighbors={k_neighbors}, autonomy_margin={knn_autonomy_margin}")
    
    nodos_ids = datos['nodos_ids']
    coords = np.array([[datos['nodos'][nid]['lat'], datos['nodos'][nid]['lon']] 
                       for nid in nodos_ids])
    
    # Obtener pares KNN
    knn_pairs = obtener_knn_pairs(coords, k_neighbors)
    print(f"  • KNN generó {len(knn_pairs)} arcos candidatos")
    
    # Construir arcos factibles por vehículo
    feasible_arcs_per_vehicle = {}
    clientes_sin_arcos = set(datos['clientes']['StandardizedID'])
    vehiculos_sin_clientes = set(datos['vehiculos']['StandardizedID'])
    
    for _, vehiculo in datos['vehiculos'].iterrows():
        vid = vehiculo['StandardizedID']
        v_tipo = vehiculo['VehicleType']
        v_capacidad = vehiculo['Capacity']
        v_autonomia = vehiculo['Range'] * knn_autonomy_margin
        
        arcos_validos = set()
        
        for i, j in knn_pairs:
            id_i = nodos_ids[i]
            id_j = nodos_ids[j]
            
            distancia = datos['distancias'][i][j]
            
            # Validar autonomía
            if distancia > v_autonomia:
                continue
            
            # Validar restricción de tipo de vehículo
            if datos['nodos'][id_j]['tipo'] == 'cliente':
                restriccion = datos['nodos'][id_j]['restriccion']
                
                if restriccion != 'any':
                    jerarquia = {'small van': 1, 'medium van': 2, 'light truck': 3}
                    if jerarquia.get(v_tipo, 0) > jerarquia.get(restriccion, 0):
                        continue
                
                demanda = datos['nodos'][id_j]['demanda']
                if demanda > v_capacidad:
                    continue
            
            arcos_validos.add((i, j))
            
            if datos['nodos'][id_j]['tipo'] == 'cliente':
                clientes_sin_arcos.discard(id_j)
                vehiculos_sin_clientes.discard(vid)
        
        feasible_arcs_per_vehicle[vid] = arcos_validos
    
    report = {
        'clientes_sin_arcos': list(clientes_sin_arcos),
        'vehiculos_sin_clientes': list(vehiculos_sin_clientes),
        'arcos_por_vehiculo': {vid: len(arcos) for vid, arcos in feasible_arcs_per_vehicle.items()}
    }
    
    print(f"\n📊 Resultados de validación:")
    print(f"  • Clientes sin arcos: {len(report['clientes_sin_arcos'])}")
    print(f"  • Vehículos sin clientes: {len(report['vehiculos_sin_clientes'])}")
    print(f"  • Arcos promedio por vehículo: {np.mean(list(report['arcos_por_vehiculo'].values())):.1f}")
    
    if report['clientes_sin_arcos']:
        print(f"\n⚠ ADVERTENCIA: {len(report['clientes_sin_arcos'])} clientes no son alcanzables")
        print(f"  Sugerencia: Aumentar k_neighbors o knn_autonomy_margin")
    
    if report['vehiculos_sin_clientes']:
        print(f"\n⚠ ADVERTENCIA: {len(report['vehiculos_sin_clientes'])} vehículos no tienen clientes")
    
    return {
        'feasible_arcs_per_vehicle': feasible_arcs_per_vehicle,
        'report': report,
        'nodos_ids': nodos_ids
    }

print("✓ Función de validación definida")


## 5. Modelo Relajado (Garantiza Factibilidad)

El modelo relajado ignora restricciones complejas (capacidad, flujo) para encontrar rápidamente una asignación básica de clientes a vehículos. Esto garantiza que tengamos una solución inicial factible.


In [ ]:
def construir_modelo_relajado(datos, feasible_arcs):
    """
    Construye modelo relajado que solo garantiza que cada cliente sea visitado
    
    Args:
        datos: diccionario de datos
        feasible_arcs: diccionario {vehicle_id: set((i,j))}
    
    Returns:
        modelo Pyomo
    """
    model = ConcreteModel(name="MDVRP_Relaxed")
    
    nodos_ids = datos['nodos_ids']
    clientes_ids = [nid for nid in nodos_ids if datos['nodos'][nid]['tipo'] == 'cliente']
    vehiculos_ids = list(feasible_arcs.keys())
    
    # Construir índice global de arcos
    x_index = set()
    for vid in vehiculos_ids:
        for (i, j) in feasible_arcs[vid]:
            x_index.add((vid, i, j))
    
    # Variables
    model.x = Var(x_index, domain=Binary)
    
    # Objetivo: minimizar distancia total
    def objetivo_rule(m):
        return sum(m.x[vid, i, j] * datos['distancias'][i][j] 
                   for (vid, i, j) in x_index)
    model.obj = Objective(rule=objetivo_rule, sense=minimize)
    
    # Restricción: cada cliente visitado exactamente una vez
    def visitar_cliente_rule(m, cliente_id):
        idx_cliente = nodos_ids.index(cliente_id)
        arcos_entrantes = [(vid, i, j) for (vid, i, j) in x_index if j == idx_cliente]
        
        if not arcos_entrantes:
            return Constraint.Skip
        
        return sum(m.x[vid, i, j] for (vid, i, j) in arcos_entrantes) == 1
    
    model.visitar_cliente = Constraint(clientes_ids, rule=visitar_cliente_rule)
    
    print(f"✓ Modelo relajado construido:")
    print(f"  • Variables x: {len(x_index)}")
    print(f"  • Clientes: {len(clientes_ids)}")
    print(f"  • Vehículos: {len(vehiculos_ids)}")
    
    return model

print("✓ Función de modelo relajado definida")


## 6. Modelo Balanced (Versión Ligera del Caso 3)

Este modelo implementa el MDVRP completo pero utilizando solo los arcos filtrados por KNN, lo que reduce significativamente el espacio de búsqueda.


In [ ]:
def construir_modelo_balanced(datos, feasible_arcs_per_vehicle, max_vehicles=None):
    """
    Construye modelo balanced del MDVRP con arcos filtrados
    
    Args:
        datos: diccionario de datos
        feasible_arcs_per_vehicle: dict {vehicle_id: set((i,j))}
        max_vehicles: límite máximo de vehículos (None = sin límite)
    
    Returns:
        modelo Pyomo
    """
    model = ConcreteModel(name="MDVRP_Balanced")
    
    nodos_ids = datos['nodos_ids']
    clientes_ids = [nid for nid in nodos_ids if datos['nodos'][nid]['tipo'] == 'cliente']
    depositos_ids = [nid for nid in nodos_ids if datos['nodos'][nid]['tipo'] == 'deposito']
    vehiculos_ids = list(feasible_arcs_per_vehicle.keys())
    
    # Construir índice global de arcos
    x_index = set()
    for vid in vehiculos_ids:
        for (i, j) in feasible_arcs_per_vehicle[vid]:
            x_index.add((vid, i, j))
    
    # Variables: x[v,i,j] - arco utilizado
    model.x = Var(x_index, domain=Binary)
    
    # Variables: y_dep[v,d] - vehículo v asignado a depósito d
    y_dep_index = [(vid, nodos_ids.index(did)) for vid in vehiculos_ids for did in depositos_ids]
    model.y_dep = Var(y_dep_index, domain=Binary)
    
    # Variables: y_use[v] - vehículo v es utilizado
    model.y_use = Var(vehiculos_ids, domain=Binary)
    
    # Variables: u[v,c] - carga acumulada del vehículo v al llegar al cliente c (MTZ)
    u_index = set()
    for vid in vehiculos_ids:
        for _, cliente in datos['clientes'].iterrows():
            cid = cliente['StandardizedID']
            idx_c = nodos_ids.index(cid)
            # Solo crear u si el cliente está en arcos del vehículo
            if any(j == idx_c for (v, i, j) in x_index if v == vid):
                u_index.add((vid, idx_c))
    model.u = Var(u_index, domain=NonNegativeReals)
    
    # Objetivo: minimizar costos totales
    def objetivo_rule(m):
        # Costo fijo
        costo_fijo = datos['parametros']['C_fixed'] * sum(m.y_use[vid] for vid in vehiculos_ids)
        
        # Costo de distancia
        costo_dist = datos['parametros']['C_dist'] * sum(
            m.x[vid, i, j] * datos['distancias'][i][j] 
            for (vid, i, j) in x_index
        )
        
        # Costo de tiempo
        costo_tiempo = datos['parametros']['C_time'] * sum(
            m.x[vid, i, j] * datos['tiempos'][i][j]
            for (vid, i, j) in x_index
        )
        
        # Costo de combustible
        costo_combustible = datos['parametros']['fuel_price'] * sum(
            m.x[vid, i, j] * datos['distancias'][i][j] / 
            datos['vehiculos'][datos['vehiculos']['StandardizedID'] == vid]['FuelEfficiency'].values[0]
            for (vid, i, j) in x_index
        )
        
        return costo_fijo + costo_dist + costo_tiempo + costo_combustible
    
    model.obj = Objective(rule=objetivo_rule, sense=minimize)
    
    # Restricción 1: Cada cliente visitado exactamente una vez
    def visitar_cliente_rule(m, cliente_id):
        idx_cliente = nodos_ids.index(cliente_id)
        arcos_entrantes = [(vid, i, j) for (vid, i, j) in x_index if j == idx_cliente]
        
        if not arcos_entrantes:
            return Constraint.Skip
        
        return sum(m.x[vid, i, j] for (vid, i, j) in arcos_entrantes) == 1
    
    model.visitar_cliente = Constraint(clientes_ids, rule=visitar_cliente_rule)
    
    # Restricción 2: Flujo en clientes (entrada = salida)
    def flujo_cliente_rule(m, vid, cid):
        idx_c = nodos_ids.index(cid)
        arcos_entrantes = [(v, i, j) for (v, i, j) in x_index if v == vid and j == idx_c]
        arcos_salientes = [(v, i, j) for (v, i, j) in x_index if v == vid and i == idx_c]
        
        if not arcos_entrantes or not arcos_salientes:
            return Constraint.Skip
        
        entrada = sum(m.x[v, i, j] for (v, i, j) in arcos_entrantes)
        salida = sum(m.x[v, i, j] for (v, i, j) in arcos_salientes)
        return entrada == salida
    
    model.flujo_cliente = Constraint(vehiculos_ids, clientes_ids, rule=flujo_cliente_rule)
    
    # Restricción 3: Flujo en depósitos (vinculado a y_dep)
    def flujo_deposito_rule(m, vid, did):
        idx_d = nodos_ids.index(did)
        arcos_salientes = [(v, i, j) for (v, i, j) in x_index if v == vid and i == idx_d]
        arcos_entrantes = [(v, i, j) for (v, i, j) in x_index if v == vid and j == idx_d]
        
        if not arcos_salientes and not arcos_entrantes:
            return Constraint.Skip
        
        salidas = sum(m.x[v, i, j] for (v, i, j) in arcos_salientes) if arcos_salientes else 0
        entradas = sum(m.x[v, i, j] for (v, i, j) in arcos_entrantes) if arcos_entrantes else 0
        
        # Si el vehículo está asignado al depósito, debe salir y regresar
        return salidas == entradas
    
    model.flujo_deposito = Constraint(vehiculos_ids, depositos_ids, rule=flujo_deposito_rule)
    
    # Restricción 4: y_dep vinculado a x
    def vincular_y_dep_rule(m, vid, did):
        idx_d = nodos_ids.index(did)
        arcos_desde_deposito = [(v, i, j) for (v, i, j) in x_index if v == vid and i == idx_d]
        
        if not arcos_desde_deposito:
            return Constraint.Skip
        
        return sum(m.x[v, i, j] for (v, i, j) in arcos_desde_deposito) <=                len(arcos_desde_deposito) * m.y_dep[vid, idx_d]
    
    model.vincular_y_dep = Constraint(vehiculos_ids, depositos_ids, rule=vincular_y_dep_rule)
    
    # Restricción 5: y_use = suma de y_dep
    def vincular_y_use_rule(m, vid):
        return m.y_use[vid] == sum(m.y_dep[vid, nodos_ids.index(did)] for did in depositos_ids)
    
    model.vincular_y_use = Constraint(vehiculos_ids, rule=vincular_y_use_rule)
    
    # Restricción 6: Capacidad con MTZ reducido
    def capacidad_mtz_rule(m, vid, i, j):
        if (vid, i, j) not in x_index:
            return Constraint.Skip
        
        id_i = nodos_ids[i]
        id_j = nodos_ids[j]
        
        # Solo aplicar si ambos son clientes
        if datos['nodos'][id_i]['tipo'] != 'cliente' or datos['nodos'][id_j]['tipo'] != 'cliente':
            return Constraint.Skip
        
        # Verificar que existan las variables u
        if (vid, i) not in u_index or (vid, j) not in u_index:
            return Constraint.Skip
        
        demanda_j = datos['nodos'][id_j]['demanda']
        capacidad_v = datos['vehiculos'][datos['vehiculos']['StandardizedID'] == vid]['Capacity'].values[0]
        
        # MTZ: u[v,i] + demanda_j <= u[v,j] + M*(1 - x[v,i,j])
        M = capacidad_v
        return m.u[vid, i] + demanda_j <= m.u[vid, j] + M * (1 - m.x[vid, i, j])
    
    model.capacidad_mtz = Constraint(x_index, rule=lambda m, vid, i, j: capacidad_mtz_rule(m, vid, i, j))
    
    # Restricción 7: Límites de u
    def limites_u_rule(m, vid, idx_c):
        if (vid, idx_c) not in u_index:
            return Constraint.Skip
        
        cid = nodos_ids[idx_c]
        demanda = datos['nodos'][cid]['demanda']
        capacidad = datos['vehiculos'][datos['vehiculos']['StandardizedID'] == vid]['Capacity'].values[0]
        
        return (demanda, m.u[vid, idx_c], capacidad)
    
    model.limites_u = Constraint(u_index, rule=lambda m, vid, idx_c: limites_u_rule(m, vid, idx_c))
    
    print(f"✓ Modelo balanced construido:")
    print(f"  • Variables x: {len(x_index)}")
    print(f"  • Variables u (MTZ): {len(u_index)}")
    print(f"  • Clientes: {len(clientes_ids)}")
    print(f"  • Depósitos: {len(depositos_ids)}")
    print(f"  • Vehículos: {len(vehiculos_ids)}")
    
    return model

print("✓ Función de modelo balanced definida")



## 7. Pipeline de Resolución Factible

Pipeline CBC→HiGHS para garantizar factibilidad y mejorar progresivamente la solución.


In [ ]:
def resolver_pipeline(model, time_cbc=120, time_highs=600, cbc_gap=0.10, highs_gap=0.02, tee=False):
    """
    Resuelve el modelo usando pipeline CBC → HiGHS
    
    Args:
        model: modelo Pyomo
        time_cbc: tiempo límite para CBC (segundos)
        time_highs: tiempo límite para HiGHS (segundos)
        cbc_gap: gap de optimalidad para CBC (0.10 = 10%)
        highs_gap: gap de optimalidad para HiGHS (0.02 = 2%)
        tee: mostrar salida del solver
    
    Returns:
        dict con resultados de CBC y HiGHS
    """
    
    resultados = {'cbc': {}, 'highs': {}, 'solution_obj': None}
    
    print("\n" + "="*60)
    print("FASE 1: Resolver con CBC (rápido, gap amplio)")
    print("="*60)
    
    # Resolver con CBC
    solver_cbc = SolverFactory('cbc')
    solver_cbc.options['seconds'] = time_cbc
    solver_cbc.options['ratioGap'] = cbc_gap
    solver_cbc.options['threads'] = 4
    
    t_start = time.time()
    result_cbc = solver_cbc.solve(model, tee=tee)
    t_cbc = time.time() - t_start
    
    # Analizar resultado CBC
    resultados['cbc']['tiempo'] = t_cbc
    resultados['cbc']['status'] = result_cbc.solver.status
    resultados['cbc']['condition'] = result_cbc.solver.termination_condition
    
    if result_cbc.solver.termination_condition == TerminationCondition.optimal or        result_cbc.solver.termination_condition == TerminationCondition.feasible:
        resultados['cbc']['objetivo'] = value(model.obj)
        resultados['cbc']['factible'] = True
        print(f"\n✓ CBC encontró solución factible")
        print(f"  Objetivo: ${resultados['cbc']['objetivo']:,.2f}")
        print(f"  Tiempo: {t_cbc:.1f}s")
    else:
        resultados['cbc']['factible'] = False
        print(f"\n✗ CBC no encontró solución factible")
        print(f"  Condición: {result_cbc.solver.termination_condition}")
        return resultados
    
    print("\n" + "="*60)
    print("FASE 2: Refinar con HiGHS (más tiempo, gap ajustado)")
    print("="*60)
    
    # Resolver con HiGHS para refinar
    solver_highs = SolverFactory('appsi_highs')
    solver_highs.options['time_limit'] = time_highs
    solver_highs.options['mip_rel_gap'] = highs_gap
    
    t_start = time.time()
    result_highs = solver_highs.solve(model, tee=tee)
    t_highs = time.time() - t_start
    
    # Analizar resultado HiGHS
    resultados['highs']['tiempo'] = t_highs
    resultados['highs']['status'] = result_highs.solver.status
    resultados['highs']['condition'] = result_highs.solver.termination_condition
    
    if result_highs.solver.termination_condition == TerminationCondition.optimal or        result_highs.solver.termination_condition == TerminationCondition.feasible:
        resultados['highs']['objetivo'] = value(model.obj)
        resultados['highs']['factible'] = True
        resultados['solution_obj'] = resultados['highs']['objetivo']
        
        mejora = ((resultados['cbc']['objetivo'] - resultados['highs']['objetivo']) / 
                  resultados['cbc']['objetivo'] * 100)
        
        print(f"\n✓ HiGHS refinó la solución")
        print(f"  Objetivo: ${resultados['highs']['objetivo']:,.2f}")
        print(f"  Mejora: {mejora:.2f}%")
        print(f"  Tiempo: {t_highs:.1f}s")
    else:
        print(f"\n⚠ HiGHS no pudo refinar, usando solución CBC")
        resultados['highs']['factible'] = False
        resultados['solution_obj'] = resultados['cbc']['objetivo']
    
    print("\n" + "="*60)
    print(f"SOLUCIÓN FINAL: ${resultados['solution_obj']:,.2f}")
    print(f"Tiempo total: {t_cbc + t_highs:.1f}s")
    print("="*60)
    
    return resultados

print("✓ Función de pipeline definida")


## 8. Extracción y Exportación de Solución

Funciones para extraer las rutas del modelo resuelto y exportarlas en formato CSV según los requisitos del enunciado.


In [ ]:
def extract_solution_from_model(model, datos):
    """
    Extrae rutas de la solución del modelo
    
    Args:
        model: modelo Pyomo resuelto
        datos: diccionario de datos
    
    Returns:
        dict con rutas por vehículo
    """
    nodos_ids = datos['nodos_ids']
    vehiculos_ids = datos['vehiculos']['StandardizedID'].tolist()
    
    # Extraer arcos activos
    arcos_activos = {}
    for vid in vehiculos_ids:
        arcos_activos[vid] = []
        for (v, i, j) in model.x:
            if v == vid and value(model.x[v, i, j]) > 0.5:
                arcos_activos[vid].append((i, j))
    
    # Reconstruir rutas
    rutas = {}
    for vid in vehiculos_ids:
        if not arcos_activos[vid]:
            continue
        
        # Encontrar depósito inicial
        arcos = arcos_activos[vid]
        # Buscar nodo que es depósito y tiene salida
        inicio = None
        for (i, j) in arcos:
            if datos['nodos'][nodos_ids[i]]['tipo'] == 'deposito':
                inicio = i
                break
        
        if inicio is None:
            continue
        
        # Reconstruir ruta greedy
        ruta = [inicio]
        current = inicio
        visitados = set([inicio])
        
        while True:
            # Buscar siguiente nodo
            siguiente = None
            for (i, j) in arcos:
                if i == current and j not in visitados:
                    siguiente = j
                    break
            
            if siguiente is None:
                # Buscar retorno al depósito
                for (i, j) in arcos:
                    if i == current and datos['nodos'][nodos_ids[j]]['tipo'] == 'deposito':
                        ruta.append(j)
                        break
                break
            
            ruta.append(siguiente)
            visitados.add(siguiente)
            current = siguiente
        
        # Convertir índices a IDs
        ruta_ids = [nodos_ids[idx] for idx in ruta]
        
        # Calcular métricas de la ruta
        distancia_total = sum(datos['distancias'][ruta[k]][ruta[k+1]] 
                             for k in range(len(ruta)-1))
        tiempo_total = sum(datos['tiempos'][ruta[k]][ruta[k+1]] 
                          for k in range(len(ruta)-1))
        
        # Clientes servidos y demandas
        clientes_servidos = [nodos_ids[idx] for idx in ruta 
                            if datos['nodos'][nodos_ids[idx]]['tipo'] == 'cliente']
        demandas = [datos['nodos'][cid]['demanda'] for cid in clientes_servidos]
        
        # Depósito
        deposito_id = ruta_ids[0]
        
        # Carga inicial (suma de demandas)
        carga_inicial = sum(demandas)
        
        # Costo de combustible
        fuel_eff = datos['vehiculos'][datos['vehiculos']['StandardizedID'] == vid]['FuelEfficiency'].values[0]
        costo_combustible = (distancia_total / fuel_eff) * datos['parametros']['fuel_price']
        
        rutas[vid] = {
            'ruta': ruta_ids,
            'deposito': deposito_id,
            'carga_inicial': carga_inicial,
            'clientes': clientes_servidos,
            'demandas': demandas,
            'distancia': distancia_total,
            'tiempo': tiempo_total,
            'costo_combustible': costo_combustible
        }
    
    return rutas

print("✓ Función de extracción definida")


In [ ]:
def export_verificacion_caso3(rutas, filename="verificacion_caso3.csv"):
    """
    Exporta rutas en formato CSV según especificación del Caso 3
    
    Formato: VehicleId,DepotId,InitialLoad,RouteSequence,ClientsServed,DemandsSatisfied,TotalDistance,TotalTime,FuelCost
    
    Args:
        rutas: dict con rutas por vehículo
        filename: nombre del archivo de salida
    """
    
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        
        # Header
        writer.writerow([
            'VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence', 
            'ClientsServed', 'DemandsSatisfied', 'TotalDistance', 
            'TotalTime', 'FuelCost'
        ])
        
        # Escribir rutas
        for vid, info in sorted(rutas.items()):
            route_seq = '-'.join(info['ruta'])
            clients_served = len(info['clientes'])
            demands_str = '-'.join(map(str, [int(d) for d in info['demandas']]))
            
            writer.writerow([
                vid,
                info['deposito'],
                f"{info['carga_inicial']:.1f}",
                route_seq,
                clients_served,
                demands_str,
                f"{info['distancia']:.2f}",
                f"{info['tiempo']:.2f}",
                f"{info['costo_combustible']:.0f}"
            ])
    
    print(f"\n✓ Archivo exportado: {filename}")
    print(f"  Vehículos utilizados: {len(rutas)}")
    print(f"  Clientes servidos: {sum(len(r['clientes']) for r in rutas.values())}")

print("✓ Función de exportación definida")


## 9. Ejecución Completa

Pipeline completo que ejecuta todo el proceso: validación → modelo relajado → modelo balanced → resolución → exportación.


In [ ]:
# Parámetros de configuración
K_NEIGHBORS = 10  # Número de vecinos KNN
AUTONOMY_MARGIN = 1.3  # Margen de autonomía (30% extra)
TIME_CBC = 120  # Tiempo para CBC (segundos)
TIME_HIGHS = 600  # Tiempo para HiGHS (segundos)

print("="*70)
print("EJECUCIÓN COMPLETA DEL CASO 3 - MDVRP ROBUSTO")
print("="*70)

# Paso 1: Validación previa
print("\nPASO 1: Validación de grafos y construcción de arcos factibles")
print("-" * 70)

k_actual = K_NEIGHBORS
intentos = 0
max_intentos = 3

while intentos < max_intentos:
    validacion = validar_previo(datos, k_neighbors=k_actual, knn_autonomy_margin=AUTONOMY_MARGIN)
    
    if len(validacion['report']['clientes_sin_arcos']) == 0:
        print(f"\n✓ Validación exitosa con k_neighbors={k_actual}")
        break
    else:
        print(f"\n⚠ Quedan {len(validacion['report']['clientes_sin_arcos'])} clientes sin arcos")
        k_actual += 3
        intentos += 1
        print(f"  Reintentando con k_neighbors={k_actual}...")

if len(validacion['report']['clientes_sin_arcos']) > 0:
    print(f"\n✗ ADVERTENCIA: No se pudieron conectar todos los clientes")
    print(f"  Continuando con {len(validacion['report']['clientes_sin_arcos'])} clientes no alcanzables")

# Paso 2: Construir y resolver modelo relajado
print("\n\nPASO 2: Modelo relajado para solución inicial")
print("-" * 70)

modelo_relajado = construir_modelo_relajado(datos, validacion['feasible_arcs_per_vehicle'])

print("\nResolviendo modelo relajado con CBC...")
solver_relax = SolverFactory('cbc')
solver_relax.options['seconds'] = 60
solver_relax.options['ratioGap'] = 0.15
result_relax = solver_relax.solve(modelo_relajado, tee=False)

if result_relax.solver.termination_condition == TerminationCondition.optimal or    result_relax.solver.termination_condition == TerminationCondition.feasible:
    print(f"✓ Modelo relajado resuelto")
    print(f"  Objetivo: ${value(modelo_relajado.obj):,.2f}")
else:
    print(f"✗ Modelo relajado no encontró solución")

# Paso 3: Construir y resolver modelo balanced
print("\n\nPASO 3: Modelo balanced (MDVRP completo)")
print("-" * 70)

modelo_balanced = construir_modelo_balanced(datos, validacion['feasible_arcs_per_vehicle'])

# Paso 4: Resolver con pipeline
print("\n\nPASO 4: Resolución con pipeline CBC → HiGHS")
print("-" * 70)

resultados = resolver_pipeline(
    modelo_balanced,
    time_cbc=TIME_CBC,
    time_highs=TIME_HIGHS,
    cbc_gap=0.10,
    highs_gap=0.02,
    tee=False
)

# Paso 5: Extraer y exportar solución
if resultados['solution_obj'] is not None:
    print("\n\nPASO 5: Extracción y exportación de solución")
    print("-" * 70)
    
    rutas = extract_solution_from_model(modelo_balanced, datos)
    export_verificacion_caso3(rutas, "verificacion_caso3.csv")
    
    # Mostrar resumen de rutas
    print("\n📋 Resumen de rutas:")
    for vid, info in sorted(rutas.items()):
        print(f"\n  {vid} (Depósito: {info['deposito']}):")
        print(f"    • Clientes: {len(info['clientes'])}")
        print(f"    • Distancia: {info['distancia']:.2f} km")
        print(f"    • Tiempo: {info['tiempo']:.2f} h")
        print(f"    • Carga: {info['carga_inicial']:.1f} kg")
        print(f"    • Ruta: {' → '.join(info['ruta'][:5])}{'...' if len(info['ruta']) > 5 else ''}")
    
    print("\n" + "="*70)
    print("✓ EJECUCIÓN COMPLETADA EXITOSAMENTE")
    print("="*70)
else:
    print("\n✗ No se pudo obtener una solución factible")


## 10. Notas Finales

### Archivos Generados

- **verificacion_caso3.csv**: Archivo de verificación con todas las rutas generadas

### Configuración Utilizada

- **Solver primario**: CBC (COIN-OR Branch and Cut)
- **Solver secundario**: HiGHS
- **Filtrado de arcos**: K-Nearest Neighbors (KNN)
- **Estrategia**: Prioridad en factibilidad sobre optimalidad

### Mejoras Posibles

1. **Aumentar K_NEIGHBORS**: Más arcos = mejor solución, pero más tiempo de cómputo
2. **Ajustar tiempos de solver**: Más tiempo puede encontrar mejores soluciones
3. **Heurísticas adicionales**: Implementar construcción greedy o local search
4. **Warm start**: Usar solución del modelo relajado como punto inicial

### Limitaciones Conocidas

- Los solvers abiertos (CBC/HiGHS) son más lentos que solvers comerciales como Gurobi
- El filtrado KNN puede eliminar arcos que serían parte de la solución óptima
- Gaps amplios (10%, 2%) sacrifican optimalidad por factibilidad

### Compatibilidad

Este notebook es compatible con:
- Google Colab (sin configuración adicional)
- Jupyter Notebook local
- VS Code con extensión de Jupyter

**Enunciado del caso**: `data/Caso3/README.md`
